<a href="https://colab.research.google.com/github/unt-iialab/INFO5731_Spring2020/blob/master/Assignments/INFO5731_Assignment_Two.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment Two**

In this assignment, you will try to gather text data from open data source via web scraping or API. After that you need to clean the text data and syntactic analysis of the data.

# **Question 1**

(40 points). Write a python program to collect text data from **either of the following sources** and save the data into a **csv file**:

(1) Collect all the customer reviews of the product [Apple iPhone 11](https://www.amazon.com/Apple-iPhone-11-64GB-Unlocked/dp/B07ZPKF8RG/ref=sr_1_13?dchild=1&keywords=iphone+12&qid=1631721363&sr=8-13) on amazon.

(2) Collect the top 10000 User Reviews of the film [Shang-Chi and the Legend of the Ten Rings](https://www.imdb.com/title/tt9376612/reviews?ref_=tt_sa_3) from IMDB.

(3) Collect all the reviews of the top 100 most popular software from [G2](https://www.g2.com/) or [Capterra](https://www.capterra.com/)

(4) Collect the abstracts of the top 10000 research papers by using the query [natural language processing](https://citeseerx.ist.psu.edu/search?q=natural+language+processing&submit.x=0&submit.y=0&sort=rlv&t=doc) from CiteSeerX.

(5) Collect all the information of the 904 narrators in the [Densho Digital Repository](https://ddr.densho.org/narrators/).

(6) Collect the top 10000 tweets by using hashtag ["#blacklivesmatter"](https://twitter.com/hashtag/blacklivesmatter) from Twitter. 


In [1]:
# Write your code here

from bs4 import BeautifulSoup
import requests
import pandas as pd
from IPython.display import clear_output

base_url = "https://www.amazon.in/Apple-iPhone-11-128GB-Black/product-reviews/B07XVLW7YK/ref=cm_cr_arp_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&pageNumber="
Reviewer = list()
Rating = list()
Review_Summary = list()
Review_Description = list()
for i in range(1, 1001):
  response = requests.get(base_url+str(i))
  soup = BeautifulSoup(response.text, 'html.parser')
  Name = soup.findAll("span", {"class" : "a-profile-name"})
  for i in range(5, len(Name)):
      Reviewer.append(Name[i].get_text())
  Product_Rating = soup.findAll("i", {"class" : "review-rating"})
  for i in range(2, len(Product_Rating)):
    Rating.append(float(Product_Rating[i].get_text().split()[0]))
  Review_Title = soup.findAll("a", {"class" : "review-title-content"}) 
  for i in range(0, len(Review_Title)):
    Review_Summary.append(Review_Title[i].get_text())
  Description = soup.findAll("span", {"class" : "review-text-content"})
  for i in range(0, len(Description)):
    Review_Description.append(Description[i].get_text())
  clear_output()
  print(f"processed {i}th page")
Reviewer += ["Amazon Customer" for _ in range(len(Review_Description)-len(Reviewer))]

df = pd.DataFrame({
  "Reviewer": Reviewer,
  "Rating": Rating,
  "Summary": Review_Summary,
  "Description": Review_Description
})

df.to_csv(r'reviews.csv', index= 'True')

df

processed 1000th page


,Reviewer,Rating,Summary,Description
0,Krusshna,1.0,\nDo not buy iphone or expensive product from ...,\n\n Please do not buy expensive product like...
1,Sunny Kumar,1.0,\nDon’t buy it from this seller\n,\n\n Bought the mobile from appario retail lt...
2,Sunny Kumar,5.0,\nFirst Time iPhone User Review :-)\n,\n\n The iPhone design is good and the camera...
3,Amazon Customer,1.0,\nWorst Experience Ever.!\n,\n\n My Phone is Producing Too Much Heat Even...
4,Satyapal singh,1.0,\niPhone 11\n,"\n\n Defective product,got heat up within 5 m..."
...,...,...,...,...
2415,Amazon Customer,3.0,\nCharger not working\n,\n\n After 3months use my charger is not work...
2416,Amazon Customer,4.0,\nFeatures\n,\n\n\n\n\n The media could ...
2417,Amazon Customer,5.0,\nValue for money\n,\n\n Value for money phone with good battery ...
2418,Amazon Customer,1.0,\nHanging\n,\n\n It’s very bad hanging all the time\n\n


# **Question 2**

(30 points). Write a python program to **clean the text data** you collected above and save the data in a new column in the csv file. The data cleaning steps include:

(1) Remove noise, such as special characters and punctuations.

(2) Remove numbers.

(3) Remove stopwords by using the [stopwords list](https://gist.github.com/sebleier/554280).

(4) Lowercase all texts

(5) Stemming. 

(6) Lemmatization.

In [2]:
# Write your code here

#before cleaning
print("Before Cleaning.\n", df.Description.head(), end="\n\n")

#1
df.Description = df.Description.str.replace('[^\w\s]','')
print("(1) Remove noise, such as special characters and punctuations.\n", df.Description.head(), end="\n\n")

#2
df.Description = df.Description.str.replace('\d+','')
print("(2) Remove numbers.\n", df.Description.head(), end="\n\n")

#3
import requests
response = requests.get("https://gist.githubusercontent.com/sebleier/554280/raw/7e0e4a1ce04c2bb7bd41089c9821dbcf6d0c786c/NLTK's%2520list%2520of%2520english%2520stopwords")
stopwords = [word for word in response.text.replace('\n', ' ').split()]
df.Description = df.Description.apply(lambda sent: " ".join((word for word in sent.split() if word.lower() not in stopwords)))
print("(3) Remove stopwords by using the stopwords list.\n", df.Description.head(), end="\n\n")

#4
df.Description = df.Description.apply(lambda sent: sent.lower())
print("(4) Lowercase all texts.\n", df.Description.head(), end="\n\n")

#5
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
df.Description.apply(lambda sent: " ".join([stemmer.stem(word) for word in sent.split()]))
print("(5) Stemming.\n", df.Description.head(), end="\n\n")

#6
from textblob import Word
import nltk
nltk.download('wordnet')
df.Description = df.Description.apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
print("(6) Lemmatization.\n", df.Description.head(), end="\n\n")

Before Cleaning.
 0    \n\n  Please do not buy expensive product like...
1    \n\n  Bought the mobile from appario retail lt...
2    \n\n  The iPhone design is good and the camera...
3    \n\n  My Phone is Producing Too Much Heat Even...
4    \n\n  Defective product,got heat up within 5 m...
Name: Description, dtype: object

(1) Remove noise, such as special characters and punctuations.
 0    \n\n  Please do not buy expensive product like...
1    \n\n  Bought the mobile from appario retail lt...
2    \n\n  The iPhone design is good and the camera...
3    \n\n  My Phone is Producing Too Much Heat Even...
4    \n\n  Defective productgot heat up within 5 mi...
Name: Description, dtype: object

(2) Remove numbers.
 0    \n\n  Please do not buy expensive product like...
1    \n\n  Bought the mobile from appario retail lt...
2    \n\n  The iPhone design is good and the camera...
3    \n\n  My Phone is Producing Too Much Heat Even...
4    \n\n  Defective productgot heat up within  min...
Name

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  # This is added back by InteractiveShellApp.init_path()


(3) Remove stopwords by using the stopwords list.
 0    Please buy expensive product like iphone Amazo...
1    Bought mobile appario retail ltd Mobile heatin...
2    iPhone design good camera quality awesome also...
3    Phone Producing Much Heat Even Didnt Put Sim N...
4    Defective productgot heat within minutes start...
Name: Description, dtype: object

(4) Lowercase all texts.
 0    please buy expensive product like iphone amazo...
1    bought mobile appario retail ltd mobile heatin...
2    iphone design good camera quality awesome also...
3    phone producing much heat even didnt put sim n...
4    defective productgot heat within minutes start...
Name: Description, dtype: object

(5) Stemming.
 0    please buy expensive product like iphone amazo...
1    bought mobile appario retail ltd mobile heatin...
2    iphone design good camera quality awesome also...
3    phone producing much heat even didnt put sim n...
4    defective productgot heat within minutes start...
Name: Descripti

# **Question 3**

(30 points). Write a python program to conduct **syntax and structure analysis** of the clean text you just saved above. The syntax and structure analysis includes: 

(1) Parts of Speech (POS) Tagging: Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) Constituency Parsing and Dependency Parsing: print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) Named Entity Recognition: Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

In [3]:
# Write your code here


#1
import spacy
nltk.download('punkt')
from spacy import displacy
from nltk import word_tokenize, pos_tag, pos_tag_sents
nltk.download('averaged_perceptron_tagger')
from nltk.tokenize import word_tokenize
from collections import Counter
#Tag Parts of Speech of each word in the text
pos = [nltk.pos_tag(word_tokenize(sent)) for sent in df.Description]
print(pos, end="\n\n")
#calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb)
count = [Counter(tag for words, tag in tags) for tags in pos]
print(count, end="\n\n")

#2
# !cd ~
# !wget http://nlp.stanford.edu/software/stanford-corenlp-full-2018-02-27.zip
# !unzip stanford-corenlp-full-2018-02-27.zip
# !cd stanford-corenlp-full-2018-02-27
# !java -Xmx4g -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer \
# -serverProperties StanfordCoreNLP-chinese.properties \
# -preload tokenize,ssplit,pos,lemma,ner,parse \
# -status_port 9001  -port 9001 -timeout 15000
# from nltk.parse.corenlp import CoreNLPDependencyParser
# dep_parser = CoreNLPDependencyParser(url='http://localhost:9001')
# sent = "I shot an elephant with a banana .".split()
# parses = list(dep_parser.parse(sent))
# parses[0].tree().pretty_print()
import nltk
from nltk import Production, CFG
# grammar
cgrammar = nltk.CFG.fromstring("""
S -> NP VP
VP -> V NP
PP -> P NP
NP -> NP PP | Det N | 'Peter' | 'Denver'
V -> 'prefers'
P -> 'from'
N -> 'flight'
Det -> 'the'
""")
# print grammar
print(cgrammar, end='\n')
sent = ['Peter', 'prefers', 'the', 'flight', 'from', 'Denver']
# Using Chart Parser
cparser = nltk.ChartParser(cgrammar)
for tree in cparser.parse(sent):
  print(tree, end="\n")
# drawing trees
!pip install svgling==0.3.1
import svgling
svgling.draw_tree(tree)
print("\n")


#3
import en_core_web_sm
nlp = en_core_web_sm.load()
nep = list()
for sentence in df.Description:
  doc=nlp(sentence)
  for X in doc.ents:
    if X.text and X.label_:
      nep.append((X.text, X.label_))
print(nep, end="\n")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[[('please', 'NN'), ('buy', 'VB'), ('expensive', 'JJ'), ('product', 'NN'), ('like', 'IN'), ('iphone', 'NN'), ('amazon', 'NN'), ('bad', 'JJ'), ('exp', 'NN'), ('charger', 'NN'), ('cord', 'NN'), ('missing', 'VBG'), ('sound', 'JJ'), ('quality', 'JJ'), ('issue', 'NN'), ('amazon', 'VBZ'), ('initial', 'JJ'), ('told', 'VBD'), ('charger', 'NN'), ('doesnt', 'NN'), ('come', 'VBP'), ('box', 'NN'), ('challenged', 'VBN'), ('asked', 'VBD'), ('reimburse', 'JJ'), ('cord', 'NN'), ('amount', 'NN'), ('requested', 'VBD'), ('replacement', 'NN'), ('day', 'NN'), ('called', 'VBD'), ('informed', 'JJ'), ('dont', 'NNS'), ('take', 'VBP'), ('responsibility', 'NN'), ('missing', 'VBG'), ('item', 'NN'), ('day', 'NN'), ('repl

**Write your explanations of the constituency parsing tree and dependency parsing tree here (Question 3-2):** 

In computational linguistics, parsing is the task of generating a parse tree from a given sentence. A parse tree, according to formal grammar, depicts the syntactical structure of a sentence. Constituency parsing and dependency parsing are two distinct types of grammar-based techniques. A constituency parse tree, according to context-free grammar, is a syntactical representation of a phrase. In this very hierarchical representation, sentences are divided into single phrasal elements. In a dependency parse tree, the syntax of a sentence is expressed in terms of directed edges between words.